In [3]:
import numpy as np

In [1]:
# Class for an a list of objects 
class ObjectHandler:
    def __init__(self):
        
        self.object_container = {}
        # self.archiveList = []
        
        self.rgb = None
        self.thermal = None
        self.depth = None
        self.tol = 0.12
        self.thermalCap = 130
        # self.weights = { "Bounding Box": 0.20, "Color": 0.20, "Temperature": 0.20, "Depth": 0.20, "Pose": 0.20 }# { "Bounding Box": 0.20, "Color": 0.15, "Temperature": 0.15, "Depth": 0.10, "Pose": 0.40 }
        # self.liklihoods = []
        # self.LIKETHRESH = 0.1
        
        self.csv = None
        
        # Depth Related Figures
        self.THRESH_LOW = 200 # 20cm
        self.THRESH_HIGH = 30000 # 30m
        self.MAXDISTANCE = 61.50 # Measured from calculating calc_spatials at max distance 30000mm at (512,512), and (0,0)
        
        ###### VALUE IS ONLY FOR OAK-D S2 | EXTRACTED FROM CALC.PY in stereoAI | Print value when with recording device ######
        self.monoHFOV = 1.2541936111357697
       
    
    # Calclates and angle of how far and deep an object is in an image
    def _calc_angle(self, frame, offset):
        return math.atan(math.tan(self.monoHFOV / 2.0) * offset / (frame.shape[1] / 2.0))
    
    
    # Gives the direction the object is moving. Used to determine if going off screen
    def _calc_derivative(self, obj):
        return obj.bb_history[-1][0] - obj.bb_history[-2][0]
    
    
    # Function to determine if two points are within a predetermined range with one another
    def inRange(self, p1, p2): # p1 newpoint p2 currentpoint
        yscale = 1
        
        xbool = p1[0] - self.tol <= p2[0] <= p1[0] + self.tol
        ybool = p1[1] - self.tol * yscale <= p2[1] <= p1[1] + self.tol * yscale
        
        return xbool and ybool
     
        
#     # Counts how long an object as been "alive" for
#     def addAlive(self):
#         for obj in self.ObjectList:
            
#             if len(obj.bb_history) > 1:
#                 derv = self._calc_derivative(obj)
            
#                 # If the object is going offscreen achive obj
#                 if obj.bb_history[-1][0] + derv >= 1 or obj.bb_history[-1][0] + derv <= 0:
#                     self.archiveList.append(obj)
#                     self.ObjectList.remove(obj)

#                     continue
                
#             obj.framesAlive += 1
                        

    # Method to view the mask of a particular object in the list
    def viewMask(self, objNum):
        
        # Create copy of rgb image to mask over
        c = self.rgb.copy()
        
        # Grab the mask and resize it appropriately to image
        mask = self.object_container[objNum].mask
        mask = cv2.resize(mask, (c.shape[1], c.shape[0]))
        
        # Invert mask to delete unrequired pixels
        mask = 1-mask
        c[ mask.astype(bool) ] = 0
        
        # Display image after non-object pixels were deleted
        cv2.imshow("MaskedObj", c)
        cv2.waitKey(1)
    
    
    ### Later add method to determine if distance is valid or not. Zeros etc
    # Calculate how far an object in an image is
    def calc_spatials(self, depthFrame, mask, bbox, averaging_method=np.median):
        
        center = [(bbox[0] + bbox[2])/2, (bbox[1] + bbox[3])/2]
        cmTOm = 1000

        depthFrame = depthFrame / 255 * 30000
        
        # Calculate the average depth in the mask
        non_zero_mask = (depthFrame != 0).all(axis=-1)
        averageDepth = averaging_method(depthFrame[mask.astype(bool)])#  & non_zero_mask])

        midX = int(depthFrame.shape[1] / 2) # middle of the depth img width
        midY = int(depthFrame.shape[0] / 2) # middle of the depth img height
        bb_x_pos = center[0] - midX
        bb_y_pos = center[1] - midY

        angle_x = self._calc_angle(depthFrame, bb_x_pos)
        angle_y = self._calc_angle(depthFrame, bb_y_pos)

        # xyz
        spatials = np.array([averageDepth * math.tan(angle_x)/cmTOm, -averageDepth * math.tan(angle_y)/cmTOm, averageDepth/cmTOm])
        
        return spatials
    
    
    ######  EXTRACTION FUNCTIONS ######
    
    def extract_rgb(self, rgb, m, averaging_method=np.median):
        
        b = int( averaging_method( rgb[m][:,0] ) )
        g = int( averaging_method( rgb[m][:,1] ) ) 
        r = int( averaging_method( rgb[m][:,2] ) )
    
        return np.array([b, g, r])
    
    def extract_thermal(self, thermal, m, averaging_method=np.median):
        return np.mean(thermal[m]) / 255 * self.thermalCap
        
        
    def extract(self, results):
        
        rgb = self.rgb.copy()
        thermal = self.thermal.copy()
        depth = self.depth.copy()
        
        imgShape = (rgb.shape[1], rgb.shape[0])
        m = np.array(results["mask"]).copy()
        m = cv2.resize(m, imgShape).astype(bool)
        
        color = self.extract_rgb(rgb, m)
        temperature = self.extract_thermal(thermal, m)
        spatials = self.calc_spatials(depth, m, results["bbox"].copy())
        
        return color, temperature, spatials
    
    
    ###### COMPARISON FUNCTIONS ######
    
    """
    Comparison functions were used for multi-dimensional comparison of objects, however due to time constrants
    unable to complete this functionality
    
    def distance(self, point):
        return np.sqrt(np.sum(point**2))
    
    def compare_class(self, base, unk):
        return int(base == unk)
    
    def compare_rgb(self, base, unk):
        return 1 - sum( abs(np.array(base) - np.array(unk)) ) / 765 # Max Difference Possible
    
    def compare_bb(self, base, unk):
        x1, y1 = base[0]-base[2]/2, base[1]-base[3]/2
        x2, y2 = base[0]+base[2]/2, base[1]+base[3]/2
        x3, y3 = unk[0]-unk[2]/2, unk[1]-unk[3]/2
        x4, y4 = unk[0]+unk[2]/2, unk[1]+unk[3]/2

        # Calculate area of intersection
        x_overlap = max(0, min(x2,x4) - max(x1,x3))
        y_overlap = max(0, min(y2,y4) - max(y1,y3))
        intersection_area = x_overlap * y_overlap

        # Calculate area of union
        box1_area = base[2] * base[3]
        box2_area = unk[2] * unk[3]
        union_area = box1_area + box2_area - intersection_area

        # Calculate IoU
        iou = intersection_area / union_area
        
        return iou
                       
    def compare_temperature(self, base, unk):
        return 1 - abs(base - unk) / self.thermalCap
        
    def compare_spatial(self, base, unk):
        max_distance = math.sqrt(2)
        return 1 - self.distance(base - unk) / self.MAXDISTANCE
    
    def compare_pose(self, base, unk):
        
        if type(base) == type(None) or type(unk) == type(None):
            return 1.0
        
        skeleton1 = base.copy()
        skeleton2 = unk.copy()
        # Extract x and y coordinates of joints
        
        conf1 = np.average(skeleton1[:, 2])
        conf2 = np.average(skeleton2[:, 2])
        conf = (conf1 + conf2) / 2
        
        joints1 = skeleton1[:, :2]
        joints2 = skeleton2[:, :2]

        # Calculate centroid of each skeleton
        centroid1 = np.mean(joints1, axis=0)
        centroid2 = np.mean(joints2, axis=0)

        # Subtract centroid from joint coordinates
        joints1 -= centroid1
        joints2 -= centroid2

        # Calculate SVD of joint coordinates
        U1, s1, VT1 = np.linalg.svd(joints1, full_matrices=False)
        U2, s2, VT2 = np.linalg.svd(joints2, full_matrices=False)

        # Calculate Frobenius norm of difference between U matrices
        U_diff = np.linalg.norm(U1 - U2)

        # Calculate similarity score
        score = 1 / (1 + U_diff)
        
        return score + (1-score)*(1-conf)
        
    def compare_objects(self, baseObjProp, unkObjProp):
        
        comparisonFunctions = [self.compare_class, self.compare_bb, self.compare_rgb, self.compare_temperature, self.compare_spatial, self.compare_pose]
        
        x = []
        for i, comparison in enumerate(comparisonFunctions):
            x.append(comparison(baseObjProp[i], unkObjProp[i]))
            
        return x
        
        
        
    ###### OBJECT APPEND FUNCTION ######
                     
                     
        # Goes through all detects to determine if it is an exisiting object
    def appendObjects(self, imgs, detections, masks, poses, frame):
        self.addAlive()
        self.thermal = imgs["Thermal"]
        self.depth = imgs["Depth"]
        self.rgb = imgs["RGB"]
        
        if detections == None or masks == None:
            return
        
        for det, mask, pose in zip(detections, masks, poses):
            pose = pose["keypoints"]
            
            unknownObjProperties = self.extract(det[0], det[1:], mask, pose)
            
            for obj in self.ObjectList:
                
                if obj.frame_history[-1] == frame:
                    self.liklihoods.append(0)
                    continue
                
                currentObjProperties = obj.get_properties()
                print("Unknown Object", unknownObjProperties[:5])
                print("Current Object", currentObjProperties[:5])
                
                compValues = self.compare_objects(currentObjProperties, unknownObjProperties)
                liklihood = compValues[0] * np.dot(compValues[1:], list(self.weights.values()))
                
                print("Object comparison", compValues, liklihood)
                self.liklihoods.append(liklihood)
            
            self.liklihoods.append(0)
            mostLikelyObject = np.argmax(self.liklihoods)
            print("ARGMX",self.liklihoods,  mostLikelyObject)
            print("\n")
            
            if self.liklihoods[mostLikelyObject] < self.LIKETHRESH:
                print("NEW OBJECT CREATED")
                self.ObjectList.append(Object(*unknownObjProperties, frame))
            else:
                print("REID FOUND")
                self.ObjectList[mostLikelyObject].update(*unknownObjProperties, frame)
            
            self.liklihoods = []
            
        # print("END")
        
        """
    
    def appendObjects(self, images, object_results):
        
        self.rgb = images["RGB"].copy()
        self.thermal = images["Thermal"].copy()
        self.depth = images["Depth"].copy()
        
        for obj in object_results:
            
            obj["color"], obj["temperature"], obj["spatials"] = self.extract(obj)
            
            if obj["track_id"] in self.object_container.keys():
                self.object_container[obj["track_id"]].update(obj)
                continue
            
            self.object_container[obj["track_id"]] = ComplexObject(obj)
            
            
    def createDataframe(self):
        
        master = []
        
        for i, key in enumerate(self.object_container.keys()):
            
            df = pd.DataFrame(self.object_container[key].properties)
            master.append(df)
        
        master = pd.concat(master)          
        master = master.sort_values(by = ['frame_num', 'track_id'])
        master = master.reset_index(drop=True)
        
        column_order = [ 'frame_num', 'track_id', 'class', 'bbox', 'color', 'temperature', 'spatials', 'keypoints' ]
        master = master.reindex(columns=column_order)
        
        display(master)
        
        return master
            

class ComplexObject:
    
    def __init__(self, properties):
        
        self.properties = {
            "class": None,
            "track_id": None,
            "color": [],
            "bbox" : [],
            "temperature": [],
            "spatials": [],
            "keypoints": [],
            "frame_num": []
        }
        
        self.color_id = tuple(np.random.randint([255, 255, 255]))
        self.color_id = tuple(int(c) for c in self.color_id)
        
        for key in self.properties.keys():
            if key in ["class", "track_id"]:
                self.properties[key] = int(properties[key])
                continue
            
            self.properties[key].append(properties[key])
            
                
    def update(self, properties):
        
        for key in self.properties.keys():
            if key in ["class", "track_id"]:
                continue
            # if key == "keypoints" and 
            
            self.properties[key].append(properties[key])
            

        
    def get_properties(self):
        pass
        # return [self.cls, self.bb_history[-1], self.color_history[-1], self.thermal_history[-1], self.spatial_history[-1], self.pose_history[-1]]
        
### ADD OBJECTS ONTO THE OBJECT CLASS


NameError: name 'np' is not defined